# IMPORT & SPLIT

In [1]:
using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Distributions, Random, ScikitLearn, GLM, Plots

In [ ]:
x_train = CSV.read("./data/new datasets/precipitation_daily_max/x_train.csv");
y_train = CSV.read("./data/new datasets/precipitation_daily_max/y_train.csv");
x_pred = CSV.read("./data/new datasets/precipitation_daily_max/x_pred.csv");

x_train = DataFrame(x_train)
y_train = convert(DataFrame, y_train)
x_pred = convert(DataFrame, x_pred);

# PLOT

In [ ]:
histogram(convert(Matrix, x_train))

We can observe that most data is skewed towards 0cm with some extreme cases over 2000cm (outliers)

we need to preprocess the data with a function to reduce this gap

$ f(x) = log_{10}(x+1) $

In [ ]:
x_train = DataFrame([log.(col.+1) for col = eachcol(x_train)])
display(histogram(convert(Matrix, x_train)))
display(describe(x_train, :mean, :std, :min, :q25, :median, :q75, :max))

We will try regression without normalisation

Intuition guides us to believe that the precipitations are correlated, but we will avoid PCA so that the model learns on it's own what meteoroligic stations are important for what gutter

# Logistic Regression

for 1st ouvrage only

In [ ]:
@sk_import linear_model: LogisticRegression
model = LogisticRegression(
    penalty="l2",
    random_state=234, # chosen by random dice roll, thus truly random
)
x = convert(Matrix, x_train)
y = convert(Matrix, y_train)
ScikitLearn.fit!(model, x, y)